# Adição de hino

Proposta de adição de hinos. A ideia é simples:

1. Criar arquivo de hino em Markdown, como se fosse na coletânea, com título como heading, e seções (Coro, Final, etc.) em negrito
2. Com arquivo de entrada, gerar SQL compatível com a estrutura do banco de dados, e adicionar a arquivo de migração

## 1. Leitura dos arquivos

Passo a passo

In [ ]:
from glob import glob

entrada = glob("arquivos_hinos\\*.md")
entrada

In [ ]:
with open(entrada[0], "r", encoding="utf-8") as f:
    conteudo = f.readlines()

conteudo

In [ ]:
titulo = None
coletanea = None
inicio_texto = 0
for i, linha in enumerate(conteudo):
    if linha.startswith("##"):
        coletanea = linha.replace("## ", "").strip()
        inicio_texto = i + 1
    elif linha.startswith("#"):
        titulo = linha.replace("# ", "").strip().upper()
        inicio_texto = i + 1

titulo, coletanea, inicio_texto

In [ ]:
texto = conteudo[inicio_texto:]
texto_full = []
for linha in texto:
    if linha.startswith("**"):
        texto_full.append(f"<b>{linha.split('**')[1].strip()}</b>\n")
    else:
        texto_full.append(linha)

texto_full = "".join(texto_full).strip()
print(texto_full)

In [ ]:
import re

TAGS_LITERAIS = ["TODOS", "M", "H", "T", "BIS", "VARÕES", "SERVAS", "CORO "]
TAGS_CONTROLE = [
    "ÍNDICE",
    "CORO (2X)",
    "\n\nCORO",
    "CORO\n",
    "1X",
    "2X",
    "3X",
    "4X",
    "()",
    "(TODOS)",
    "(M)",
    "(H)",
    "(T)",
    "(BIS)",
    "(VARÕES)",
    "(SERVAS)",
    "REPETIR O LOUVOR",
    "REPETIR 1ª ESTROFE",
    "FINAL:",
    "BIS NO FINAL",
    "IGREJA CRISTÃ MARANATA",
    "ATUALIZAÇÃO",
    "INSTRUMENTOS",
]


def set_text_clean(texto):
    texts_clean = [re.sub(r"\s+", " ", text) for text in texto]
    texts_clean = [line.replace("**", "") for line in texts_clean]
    texts_clean = [line for line in texts_clean if line not in TAGS_LITERAIS]

    new_texts_clean = []
    for line in texts_clean:
        for tag in TAGS_CONTROLE:
            line = line.upper().replace(tag, "")
        line = line.strip()
        if line:
            new_texts_clean.append(line)
    texts_clean = new_texts_clean
    # texts_clean = list(dict.fromkeys(texts_clean))
    texts_clean = " ".join(texts_clean)
    texts_clean = texts_clean.replace("\n", " ")
    # remove all double quotes
    texts_clean = texts_clean.replace("“", "")
    texts_clean = texts_clean.replace("”", "")
    texts_clean = texts_clean.replace('"', "")
    return texts_clean


texto_clean = set_text_clean(texto)
texto_clean

### Função

In [ ]:
def processa_texto(arquivo):
    with open(arquivo, "r", encoding="utf-8") as f:
        conteudo = f.readlines()

    titulo = None
    coletanea = None
    inicio_texto = 0
    for i, linha in enumerate(conteudo):
        if linha.startswith("##"):
            coletanea = linha.replace("## ", "").strip()
            inicio_texto = i + 1
        elif linha.startswith("#"):
            titulo = linha.replace("# ", "").strip().upper()
            inicio_texto = i + 1

    texto = conteudo[inicio_texto:]
    texto_clean = set_text_clean(texto)

    texto_full = []
    for linha in texto:
        if linha.startswith("**"):
            texto_full.append(f"<b>{linha.split('**')[1].strip()}</b>\n")
        else:
            texto_full.append(linha)

    texto_full = "".join(texto_full).strip()

    return titulo, coletanea, texto_full, texto_clean

## 2. Processamento do texto em SQL

In [ ]:
arquivo_migracao = "010-add-hinos-avulsos.sql"

with open("..\\db\\migrations\\" + arquivo_migracao, "w", encoding="utf-8") as f:
    for arquivo in entrada:
        titulo, coletanea, texto_full, texto_clean = processa_texto(arquivo)

        linha = (
            "INSERT INTO hino (numero, nome, texto, texto_limpo, coletanea_id, date_insert, date_update) VALUES ('null','"
            + titulo
            + "','"
            + texto_full.replace("\n", "\\n").replace("'", "''")
            + "','"
            + texto_clean.replace("'", "''")
            + "',(select id from coletanea where nome = '"
            + coletanea
            + "'),CURRENT_TIMESTAMP,CURRENT_TIMESTAMP);\n"
        )

        f.write(linha)